In [461]:
import pandas as pd
import numpy as np
#from sklearn import preprocessing

df_basinc = pd.read_csv('/Users/ardahanaytan/Desktop/STAJ/Hava Durumu Tahmin Projesi/data/weather_datas/basinc.csv')
df_bulut = pd.read_csv('/Users/ardahanaytan/Desktop/STAJ/Hava Durumu Tahmin Projesi/data/weather_datas/bulut.csv')
df_nem = pd.read_csv('/Users/ardahanaytan/Desktop/STAJ/Hava Durumu Tahmin Projesi/data/weather_datas/nem.csv')
df_ruzgar = pd.read_csv('/Users/ardahanaytan/Desktop/STAJ/Hava Durumu Tahmin Projesi/data/weather_datas/ruzgar_hizi_yonu.csv')
df_sicaklik = pd.read_csv('/Users/ardahanaytan/Desktop/STAJ/Hava Durumu Tahmin Projesi/data/weather_datas/sicaklik.csv')
df_yagmur = pd.read_csv('/Users/ardahanaytan/Desktop/STAJ/Hava Durumu Tahmin Projesi/data/weather_datas/yagmur.csv')
df_kar = pd.read_csv('/Users/ardahanaytan/Desktop/STAJ/Hava Durumu Tahmin Projesi/data/weather_datas/kar.csv')

ctable = ['Basinc','Bulut','Nem','Ruzgar Hizi','Ruzgar Yonu','Sicaklik','Yagis','Kar']
tables = [df_nem, df_ruzgar, df_sicaklik, df_yagmur, df_kar]
real_df = pd.merge(df_basinc, df_bulut, on = ['Yil', 'Ay', 'Gun', 'Saat'])

for i in tables:
    real_df = pd.merge(real_df, i, on = ['Yil', 'Ay', 'Gun', 'Saat'])

df = real_df.copy()
df = df.dropna()

df['Yil'] = df['Yil'].str[1:].astype(int)
df['Ay'] = df['Ay'].map({
    'Ocak': 1, 'Şubat': 2, 'Mart': 3, 'Nisan': 4, 'Mayıs': 5, 'Haziran': 6,
    'Temmuz': 7, 'Ağustos': 8, 'Eylül': 9, 'Ekim': 10, 'Kasım': 11, 'Aralık': 12
})
df['Gun'] = df['Gun'].str[1:].astype(int)
df['Saat'] = df['Saat'].str[1:].str[:2].astype(int)

df['datetime'] = pd.to_datetime(df[['Yil', 'Ay', 'Gun']].astype(str).agg('-'.join, axis=1) + ' ' + df['Saat'].astype(str), format='%Y-%m-%d %H')
df.set_index('datetime', inplace=True)
df.drop(columns=['Yil', 'Ay', 'Gun', 'Saat'], inplace=True)
df.columns = ctable

#df = pd.DataFrame(preprocessing.scale(df.select_dtypes('float64')), columns=ctable)

df['hedef'] = df.shift(-24)['Basinc']
df.iloc[-24:] = df.iloc[-48:-24]

In [462]:
df.index

DatetimeIndex(['2023-01-01 00:00:00', '2023-01-01 01:00:00',
               '2023-01-01 02:00:00', '2023-01-01 03:00:00',
               '2023-01-01 04:00:00', '2023-01-01 05:00:00',
               '2023-01-01 06:00:00', '2023-01-01 07:00:00',
               '2023-01-01 08:00:00', '2023-01-01 09:00:00',
               ...
               '2024-08-15 14:00:00', '2024-08-15 15:00:00',
               '2024-08-15 16:00:00', '2024-08-15 17:00:00',
               '2024-08-15 18:00:00', '2024-08-15 19:00:00',
               '2024-08-15 20:00:00', '2024-08-15 21:00:00',
               '2024-08-15 22:00:00', '2024-08-15 23:00:00'],
              dtype='datetime64[ns]', name='datetime', length=14232, freq=None)

In [463]:
df.tail()

,Basinc,Bulut,Nem,Ruzgar Hizi,Ruzgar Yonu,Sicaklik,Yagis,Kar,hedef
datetime,,,,,,,,,
2024-08-15 19:00:00,1015.3,28.200000,71.347370,30.625402,252.20401,22.820246,0.0,0.0,1017.0
2024-08-15 20:00:00,1016.1,27.900002,78.545204,28.732420,254.74487,21.200245,0.0,0.0,1017.7
2024-08-15 21:00:00,1016.4,29.100000,84.666410,26.717722,255.96376,20.060246,0.0,0.0,1018.0
2024-08-15 22:00:00,1017.1,39.000000,83.306114,13.708390,240.06850,20.290245,0.0,0.0,1018.5
2024-08-15 23:00:00,1016.9,31.000000,85.810330,15.124284,231.76617,19.600245,0.0,0.0,1018.4


In [464]:
df.corr()

,Basinc,Bulut,Nem,Ruzgar Hizi,Ruzgar Yonu,Sicaklik,Yagis,Kar,hedef
Basinc,1.000000,-0.177095,0.041267,-0.285506,-0.099015,-0.192603,-0.227333,-0.082160,0.782354
Bulut,-0.177095,1.000000,0.308911,0.323632,0.086428,-0.289836,0.325553,0.100265,-0.053453
Nem,0.041267,0.308911,1.000000,0.110925,0.031906,-0.633379,0.188131,0.072887,0.067576
Ruzgar Hizi,-0.285506,0.323632,0.110925,1.000000,0.179350,-0.092156,0.243736,-0.003365,-0.206394
Ruzgar Yonu,-0.099015,0.086428,0.031906,0.179350,1.000000,0.087922,0.052070,0.018977,-0.128937
Sicaklik,-0.192603,-0.289836,-0.633379,-0.092156,0.087922,1.000000,-0.012091,-0.120737,-0.164974
Yagis,-0.227333,0.325553,0.188131,0.243736,0.052070,-0.012091,1.000000,0.105428,-0.102919
Kar,-0.082160,0.100265,0.072887,-0.003365,0.018977,-0.120737,0.105428,1.000000,0.000655
hedef,0.782354,-0.053453,0.067576,-0.206394,-0.128937,-0.164974,-0.102919,0.000655,1.000000


In [465]:
df.isnull().any()

Basinc         False
Bulut          False
Nem            False
Ruzgar Hizi    False
Ruzgar Yonu    False
Sicaklik       False
Yagis          False
Kar            False
hedef          False
dtype: bool

In [466]:
predictors = df.columns[~df.columns.isin(['hedef'])]
predictors

Index(['Basinc', 'Bulut', 'Nem', 'Ruzgar Hizi', 'Ruzgar Yonu', 'Sicaklik',
       'Yagis', 'Kar'],
      dtype='object')

In [467]:
from sklearn.linear_model import Ridge
rr = Ridge(alpha=0.1)

In [468]:
len(df.index)

14232

In [469]:
def backtest(weather, model, predictors, start = 14200, step = 12):
    all_predictions = []
    
    for i in range(start, weather.shape[0], step):
        train = weather.iloc[:i,:]
        test = weather.iloc[i:(i+step),:]
        
        model.fit(train[predictors], train['hedef'])
        
        preds = model.predict(test[predictors])
        
        preds = pd.Series(preds, index=test.index)
        combined = pd.concat([test['hedef'], preds], axis=1)
        
        combined.columns = ['actual','prediction']
        combined['diff'] = (combined['prediction'] - combined['actual']).abs()
        
        all_predictions.append(combined)
    return pd.concat(all_predictions)

In [470]:
predictions = backtest(df, rr, predictors)

In [471]:
predictions

,actual,prediction,diff
datetime,,,
2024-08-14 16:00:00,1016.3,1013.647765,2.652235
2024-08-14 17:00:00,1016.3,1013.903124,2.396876
2024-08-14 18:00:00,1016.6,1014.333995,2.266005
2024-08-14 19:00:00,1017.0,1015.099468,1.900532
2024-08-14 20:00:00,1017.7,1015.746041,1.953959
2024-08-14 21:00:00,1018.0,1016.034277,1.965723
2024-08-14 22:00:00,1018.5,1016.901232,1.598768
2024-08-14 23:00:00,1018.4,1016.626577,1.773423
2024-08-15 00:00:00,1016.7,1012.636893,4.063107


In [472]:
predictions['diff'].mean()

3.321327862359716

In [473]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(predictions['actual'], predictions['prediction'])

3.321327862359716

In [474]:
def pct_diff(old, new):
    return (new - old) / old

def compute_rolling(weather, horizon, col):
    label = f"rolling_{horizon}_{col}"
    weather[label] = weather[col].rolling(horizon).mean()
    weather[f"{label}_pct"] = pct_diff(weather[label], weather[col])
    return weather

In [475]:
rolling_horizons = [3,24]

In [476]:
for horizon in rolling_horizons:
    for col in ['Basinc','Bulut','Nem','Ruzgar Hizi','Ruzgar Yonu','Sicaklik','Yagis','Kar']:
        df = compute_rolling(df, horizon, col)

In [477]:
df

,Basinc,Bulut,Nem,Ruzgar Hizi,Ruzgar Yonu,Sicaklik,Yagis,Kar,hedef,rolling_3_Basinc,...,rolling_24_Ruzgar Hizi,rolling_24_Ruzgar Hizi_pct,rolling_24_Ruzgar Yonu,rolling_24_Ruzgar Yonu_pct,rolling_24_Sicaklik,rolling_24_Sicaklik_pct,rolling_24_Yagis,rolling_24_Yagis_pct,rolling_24_Kar,rolling_24_Kar_pct
datetime,,,,,,,,,,,,,,,,,,,,,
2023-01-01 00:00:00,1021.4,8.700001,80.826390,83.567314,230.06802,12.270246,0.0,0.0,1020.4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-01 01:00:00,1021.3,22.000000,79.099785,87.857704,229.65335,12.340245,0.0,0.0,1020.1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-01 02:00:00,1021.5,44.000000,80.655174,90.806390,230.63068,11.840245,0.0,0.0,1019.8,1021.400000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-01 03:00:00,1021.7,22.000000,81.831300,90.299736,230.66238,11.550245,0.0,0.0,1019.3,1021.500000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-01 04:00:00,1021.9,4.000000,81.179360,90.745705,231.76617,11.260245,0.0,0.0,1019.2,1021.700000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-15 19:00:00,1015.3,28.200000,71.347370,30.625402,252.20401,22.820246,0.0,0.0,1017.0,1014.400000,...,33.850418,-0.095273,212.388954,0.187463,23.705245,-0.037333,0.041667,-1.0,0.0,NaN
2024-08-15 20:00:00,1016.1,27.900002,78.545204,28.732420,254.74487,21.200245,0.0,0.0,1017.7,1015.200000,...,33.850418,-0.151195,212.388954,0.199426,23.705245,-0.105673,0.041667,-1.0,0.0,NaN
2024-08-15 21:00:00,1016.4,29.100000,84.666410,26.717722,255.96376,20.060246,0.0,0.0,1018.0,1015.933333,...,33.850418,-0.210712,212.388954,0.205165,23.705245,-0.153763,0.041667,-1.0,0.0,NaN


In [478]:
df = df.iloc[24:, :]

In [479]:
df

,Basinc,Bulut,Nem,Ruzgar Hizi,Ruzgar Yonu,Sicaklik,Yagis,Kar,hedef,rolling_3_Basinc,...,rolling_24_Ruzgar Hizi,rolling_24_Ruzgar Hizi_pct,rolling_24_Ruzgar Yonu,rolling_24_Ruzgar Yonu_pct,rolling_24_Sicaklik,rolling_24_Sicaklik_pct,rolling_24_Yagis,rolling_24_Yagis_pct,rolling_24_Kar,rolling_24_Kar_pct
datetime,,,,,,,,,,,,,,,,,,,,,
2023-01-02 00:00:00,1020.4,15.900001,87.687210,60.110300,235.73890,8.490246,0.0,0.0,1029.6,1020.300000,...,70.689924,-0.149662,232.195086,0.015262,11.880245,-0.285348,0.000000,NaN,0.0,NaN
2023-01-02 01:00:00,1020.1,17.100000,88.342500,61.773310,229.25385,8.340245,0.0,0.0,1029.7,1020.300000,...,69.603075,-0.112492,232.178440,-0.012596,11.713579,-0.287985,0.000000,NaN,0.0,NaN
2023-01-02 02:00:00,1019.8,51.000000,88.970924,64.912900,225.22469,7.840245,0.0,0.0,1029.7,1020.100000,...,68.524179,-0.052701,231.953190,-0.029008,11.546912,-0.321009,0.000000,NaN,0.0,NaN
2023-01-02 03:00:00,1019.3,88.000000,88.475746,67.781410,222.41698,7.820245,0.0,0.0,1029.9,1019.733333,...,67.585916,0.002893,231.609632,-0.039690,11.391495,-0.313501,0.000000,NaN,0.0,NaN
2023-01-02 04:00:00,1019.2,61.000000,87.307465,68.777900,227.12110,7.760245,0.0,0.0,1029.6,1019.433333,...,66.670590,0.031608,231.416088,-0.018560,11.245662,-0.309934,0.000000,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-15 19:00:00,1015.3,28.200000,71.347370,30.625402,252.20401,22.820246,0.0,0.0,1017.0,1014.400000,...,33.850418,-0.095273,212.388954,0.187463,23.705245,-0.037333,0.041667,-1.0,0.0,NaN
2024-08-15 20:00:00,1016.1,27.900002,78.545204,28.732420,254.74487,21.200245,0.0,0.0,1017.7,1015.200000,...,33.850418,-0.151195,212.388954,0.199426,23.705245,-0.105673,0.041667,-1.0,0.0,NaN
2024-08-15 21:00:00,1016.4,29.100000,84.666410,26.717722,255.96376,20.060246,0.0,0.0,1018.0,1015.933333,...,33.850418,-0.210712,212.388954,0.205165,23.705245,-0.153763,0.041667,-1.0,0.0,NaN


In [480]:
df = df.fillna(0)
df

,Basinc,Bulut,Nem,Ruzgar Hizi,Ruzgar Yonu,Sicaklik,Yagis,Kar,hedef,rolling_3_Basinc,...,rolling_24_Ruzgar Hizi,rolling_24_Ruzgar Hizi_pct,rolling_24_Ruzgar Yonu,rolling_24_Ruzgar Yonu_pct,rolling_24_Sicaklik,rolling_24_Sicaklik_pct,rolling_24_Yagis,rolling_24_Yagis_pct,rolling_24_Kar,rolling_24_Kar_pct
datetime,,,,,,,,,,,,,,,,,,,,,
2023-01-02 00:00:00,1020.4,15.900001,87.687210,60.110300,235.73890,8.490246,0.0,0.0,1029.6,1020.300000,...,70.689924,-0.149662,232.195086,0.015262,11.880245,-0.285348,0.000000,0.0,0.0,0.0
2023-01-02 01:00:00,1020.1,17.100000,88.342500,61.773310,229.25385,8.340245,0.0,0.0,1029.7,1020.300000,...,69.603075,-0.112492,232.178440,-0.012596,11.713579,-0.287985,0.000000,0.0,0.0,0.0
2023-01-02 02:00:00,1019.8,51.000000,88.970924,64.912900,225.22469,7.840245,0.0,0.0,1029.7,1020.100000,...,68.524179,-0.052701,231.953190,-0.029008,11.546912,-0.321009,0.000000,0.0,0.0,0.0
2023-01-02 03:00:00,1019.3,88.000000,88.475746,67.781410,222.41698,7.820245,0.0,0.0,1029.9,1019.733333,...,67.585916,0.002893,231.609632,-0.039690,11.391495,-0.313501,0.000000,0.0,0.0,0.0
2023-01-02 04:00:00,1019.2,61.000000,87.307465,68.777900,227.12110,7.760245,0.0,0.0,1029.6,1019.433333,...,66.670590,0.031608,231.416088,-0.018560,11.245662,-0.309934,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-15 19:00:00,1015.3,28.200000,71.347370,30.625402,252.20401,22.820246,0.0,0.0,1017.0,1014.400000,...,33.850418,-0.095273,212.388954,0.187463,23.705245,-0.037333,0.041667,-1.0,0.0,0.0
2024-08-15 20:00:00,1016.1,27.900002,78.545204,28.732420,254.74487,21.200245,0.0,0.0,1017.7,1015.200000,...,33.850418,-0.151195,212.388954,0.199426,23.705245,-0.105673,0.041667,-1.0,0.0,0.0
2024-08-15 21:00:00,1016.4,29.100000,84.666410,26.717722,255.96376,20.060246,0.0,0.0,1018.0,1015.933333,...,33.850418,-0.210712,212.388954,0.205165,23.705245,-0.153763,0.041667,-1.0,0.0,0.0


In [481]:
def expand_mean(df):
    return df.expanding(1).mean()

for col in ['Basinc','Bulut','Nem','Ruzgar Hizi','Ruzgar Yonu','Sicaklik','Yagis','Kar']:
    df[f'month_avg_{col}'] = df[col].groupby(df.index.month, group_keys=False).apply(expand_mean)
    df[f'day_avg_{col}'] = df[col].groupby(df.index.day_of_year, group_keys=False).apply(expand_mean)
    df[f'hour_avg_{col}'] = df[col].groupby(df.index.hour, group_keys=False).apply(expand_mean)

In [482]:
df

,Basinc,Bulut,Nem,Ruzgar Hizi,Ruzgar Yonu,Sicaklik,Yagis,Kar,hedef,rolling_3_Basinc,...,hour_avg_Ruzgar Yonu,month_avg_Sicaklik,day_avg_Sicaklik,hour_avg_Sicaklik,month_avg_Yagis,day_avg_Yagis,hour_avg_Yagis,month_avg_Kar,day_avg_Kar,hour_avg_Kar
datetime,,,,,,,,,,,,,,,,,,,,,
2023-01-02 00:00:00,1020.4,15.900001,87.687210,60.110300,235.73890,8.490246,0.0,0.0,1029.6,1020.300000,...,235.738900,8.490246,8.490246,8.490246,0.000000,0.000000,0.000000,0.0,0.0,0.000000
2023-01-02 01:00:00,1020.1,17.100000,88.342500,61.773310,229.25385,8.340245,0.0,0.0,1029.7,1020.300000,...,229.253850,8.415246,8.415246,8.340245,0.000000,0.000000,0.000000,0.0,0.0,0.000000
2023-01-02 02:00:00,1019.8,51.000000,88.970924,64.912900,225.22469,7.840245,0.0,0.0,1029.7,1020.100000,...,225.224690,8.223579,8.223579,7.840245,0.000000,0.000000,0.000000,0.0,0.0,0.000000
2023-01-02 03:00:00,1019.3,88.000000,88.475746,67.781410,222.41698,7.820245,0.0,0.0,1029.9,1019.733333,...,222.416980,8.122745,8.122745,7.820245,0.000000,0.000000,0.000000,0.0,0.0,0.000000
2023-01-02 04:00:00,1019.2,61.000000,87.307465,68.777900,227.12110,7.760245,0.0,0.0,1029.6,1019.433333,...,227.121100,8.050245,8.050245,7.760245,0.000000,0.000000,0.000000,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-15 19:00:00,1015.3,28.200000,71.347370,30.625402,252.20401,22.820246,0.0,0.0,1017.0,1014.400000,...,227.090366,21.987391,24.582064,13.337239,0.086091,0.038636,0.114865,0.0,0.0,0.002956
2024-08-15 20:00:00,1016.1,27.900002,78.545204,28.732420,254.74487,21.200245,0.0,0.0,1017.7,1015.200000,...,226.590362,21.986676,24.506912,12.391951,0.086013,0.037778,0.105574,0.0,0.0,0.002128
2024-08-15 21:00:00,1016.4,29.100000,84.666410,26.717722,255.96376,20.060246,0.0,0.0,1018.0,1015.933333,...,228.191339,21.984928,24.410245,11.692357,0.085935,0.036957,0.099155,0.0,0.0,0.002010


In [483]:
predictors = df.columns[~df.columns.isin(['hedef'])]
predictions = backtest(df, rr, predictors)
predictions['diff'].mean()

0.7777477576345717

In [484]:
predictions

,actual,prediction,diff
datetime,,,
2024-08-15 16:00:00,1016.3,1015.330998,0.969002
2024-08-15 17:00:00,1016.3,1016.163624,0.136376
2024-08-15 18:00:00,1016.6,1016.195833,0.404167
2024-08-15 19:00:00,1017.0,1018.180301,1.180301
2024-08-15 20:00:00,1017.7,1018.622410,0.922410
2024-08-15 21:00:00,1018.0,1017.131296,0.868704
2024-08-15 22:00:00,1018.5,1018.506721,0.006721
2024-08-15 23:00:00,1018.4,1016.665699,1.734301
